# Using python

```{note}
None of the following information is true. The python env should works as long as the python version is compiled with x86-64 instruction set. However, some compute nodes still throws "Illegal instruction" error. I have to still uncover the reason behind it.
```

I never thought that using python on a HPC cluster might be a problem, and I will ever need to document it. CSD3 contains partitions with multiple CPUs brands. The Ampere GPU nodes runs of a AMD CPU and the login nodes run on Intel CPU. So, python compiled for a Intel CPUs will not work on AMD CPUs because of Intel-specific instructions (like AVX-512) don’t exist on AMD EPYC. So, if you created a virtual environment on the login node and tried to run it on the compute node, it will not work and give you the following error:

```sh
(.venv) [ir-shar8@gpu-q-8 ~]$ python
Illegal instruction
```

However, Dr. Lisa made sure that AMD CPUs are fully compatible with Intel’s x86-64 instruction set. That is why when the virtual environment is created on the compute node, it will work on the login node. To check the CPU brand, you can use the following command:

```sh
cat /proc/cpuinfo | grep "model name"
```

## Using python compiled with AMD CPU
CSD3 has bare-metal python, virtualenv, anaconda and miniconda installed as modules. But, all of them are very old and won't work with the NVIDIA driver 550 compiled against CUDA 12.4. Sure one can just download miniconda from internet, but if you lazy like me, just use the following module.

```sh
module load python/3.11.0-icl 
```
This is only new python version which is compatible with the NVIDIA driver we have on ampere partitions. Remember to create a virtual environment on the compute node not on the login node. The compute nodes have internet access, so you can download the packages you need.

## Summary
* Python compiled on AMD = Universally compatible.
* Python compiled on Intel = Not always AMD-compatible (because of AVX-512).

## Example
Virtual environment running on the login node:
```sh
[ir-shar8@login-p-1 ~]$ source .venv/bin/activate
(.venv) [ir-shar8@login-p-1 ~]$ python
Python 3.11.11 (main, Dec 12 2024, 16:28:23) [GCC 8.5.0 20210514 (Red Hat 8.5.0-22)] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> torch.cuda.is_available()
False
>>> exit()
```

Virtual environment running on the compute node:
```sh
(.venv) [ir-shar8@gpu-q-17 ~]$ python
Python 3.11.11 (main, Dec 12 2024, 16:28:23) [GCC 8.5.0 20210514 (Red Hat 8.5.0-22)] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> torch.cuda.get_device_name(0)
'NVIDIA A100-SXM4-80GB'
>>> 
```

There is not "illegal instruction" error on either of the nodes.